In [55]:
import json
import networkx as nx

# Cargar el archivo de la jerarquía JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as f:
    grafo_json = json.load(f)

# Función para realizar una consulta simple basada en la estructura actual
def realizar_consulta(grafo_json, consulta):
    resultados = []
    
    # Buscar en los cultivos y productos
    for cultivo, data in grafo_json.items():
        if consulta.lower() in cultivo.lower():
            resultados.append(f"Encontrado en el cultivo: {cultivo}")
        
        if 'Productos' in data:
            for producto, info_producto in data['Productos'].items():
                if consulta.lower() in producto.lower():
                    resultados.append(f"Encontrado en el producto: {producto} del cultivo {cultivo}")
                
                if 'Controla a' in info_producto:
                    for plaga, detalles in info_producto['Controla a'].items():
                        if consulta.lower() in plaga.lower() or any(consulta.lower() in momento.lower() for momento in detalles['Momento de aplicacion']):
                            resultados.append(f"Producto: {producto} en cultivo {cultivo} controla a la plaga {plaga}")
                            for momento in detalles['Momento de aplicacion']:
                                resultados.append(f"    Momento de aplicacion: {momento}")
    
    return resultados



In [56]:
# Ejemplo de consulta
consulta = "Opera®"
resultados = realizar_consulta(grafo_json, consulta)

# Mostrar los resultados
if resultados:
    for resultado in resultados:
        print(resultado)
else:
    print("No se encontraron resultados.")

Encontrado en el producto: Opera®  del cultivo Maíz
Encontrado en el producto: Opera®  del cultivo Soja
Encontrado en el producto: Opera®  del cultivo Girasol
Encontrado en el producto: Opera®  del cultivo Trigo
Encontrado en el producto: Opera®  del cultivo Maní
Encontrado en el producto: Opera®  del cultivo Ajo
Encontrado en el producto: Opera®  del cultivo Garbanzo


In [58]:
def buscar_categoria(categorias, busqueda):
    resultados = {}
    print(f"Buscando: {busqueda}")  # Imprimir el término que se está buscando
    for cultivo, datos_cultivo in categorias.items():
        # Filtrar también por el nombre del cultivo
        if busqueda.lower() in cultivo.lower():
            print(f"Encontrado cultivo: {cultivo}")
            if 'Productos' in datos_cultivo:
                for producto, info_producto in datos_cultivo['Productos'].items():
                    if busqueda.lower() in producto.lower():  # Verifica si el término está en el producto
                        print(f"Encontrado producto: {producto}")
                    # Verifica si el producto contiene "Controla a"
                    if 'Controla a' in info_producto:
                        for plaga, detalles in info_producto['Controla a'].items():
                            if busqueda.lower() in plaga.lower() or any(busqueda.lower() in momento.lower() for momento in detalles['Momento de aplicacion']):
                                if cultivo not in resultados:
                                    resultados[cultivo] = {}
                                if producto not in resultados[cultivo]:
                                    resultados[cultivo][producto] = {}
                                resultados[cultivo][producto][plaga] = detalles['Momento de aplicacion']
                    else:
                        print(f"  El producto no tiene 'Controla a': {producto}")

    # Imprimir los resultados
    if resultados:
        for cultivo, productos in resultados.items():
            print(f"Cultivo: {cultivo}")
            for producto, plagas in productos.items():
                print(f"  Producto: {producto}")
                for plaga, momentos in plagas.items():
                    print(f"    Plaga: {plaga}")
                    for momento in momentos:
                        print(f"      {momento}")
    else:
        print("No se encontraron resultados completos para:", busqueda)
        

buscar_categoria(grafo_json, 'Soja')


Buscando: Soja
Encontrado cultivo: Soja
Cultivo: Soja
  Producto: Fastac®
    Plaga: Nezara viridula
      Aplicar, de acuerdo a los umbrales establecidos por INTA, a partir de 0,4 chinches por metro lineal, cuando la soja esté en estado de formación de vainas (R3). Se recomienda trabajar con altos volúmenes que aseguren una buena penetración en el canopeo, logrando al menos 20 gotas/cm2 en la parte media de la planta.
      A partir de la floración cuando se observen 2 chinches por metro lineal de surco. Mezclar con 500 cc de endosulfán 35%.
      Comenzar aplicaciones al inicio de ataque, siguiendo los umbrales establecidos por INTA, desde 0,4 chinches/m (adultos + ninfas mayores a 0,5 cm), cuando el cultivo esté en inicio de formación de vainas (R3). No realizar más de dos aplicaciones por ciclo de cultivo.
  Producto: CZ 3.45 S STS
    Plaga: Phytophthora sojae
  Producto: CZ 3.45 S STS I
    Plaga: Phytophthora sojae
  Producto: CZ 3.85 S
    Plaga: Phytophthora sojae
  Producto: 